## [수정X] 캐글 사용을 위한 설정

In [13]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 66 Oct 31 10:56 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=ffe24cc5c3e11460d3305fe6cc888219f83c0bbaf408d0f27ef47b21d90d0912
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


## [수정X]
### [주의] 반드시 Token 다운 로드 후 업로드


In [14]:

# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v


Kaggle API 1.5.6


## [수정X] 데이터 셋 다운로드 후 압축 풀기

In [15]:
! kaggle competitions download -c 2019-fall-pr-project

import zipfile
import os

os.mkdir("/content/input2")

zip_ref = zipfile.ZipFile("/content/2019-fall-pr-project.zip", 'r')
zip_ref.extractall("/content/input2")
zip_ref.close()

 98% 539M/548M [00:03<00:00, 203MB/s]
100% 548M/548M [00:03<00:00, 163MB/s]


## [수정O] 라이브러리 로드
- 필요시 라이브러리 추가 가능

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from skimage.feature import hog
from skimage.color import rgb2grey

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, auc
# 5를 해보고 실행에 옮겨봄..
from sklearn.preprocessing import StandardScaler


## [수정O] 학습 데이터 준비 
- 영상 32x32 로 Resize 후 1D Vector 로 변환 할 것
- 코드 검증이 끝나기 전까지는 전체 데이터를 학습에 사용하지 말고 샘플만 사용하여 학습
- 코드 검증 후 전체 데이터 셋을 활용하여 학습 & 테스트

In [0]:
scaler = StandardScaler()
datatrain = "./input2/train/train/"
train_db = glob(datatrain+'*.*.jpg')

train = list()
label = list()

for i in range(len(train_db)):

  if train_db[i][21] == 'd' :
    label.append(0)
  else :
    label.append(1)
  img = Image.open(train_db[i])
  img = img.resize((32,32))
  aumg = np.array(img)/255

  aumg = aumg.reshape(-1)
  train.append(aumg)


In [0]:
train = np.array(train) # 1 d vector
scaler.fit(train)
train = scaler.transform(train)
label = np.array(label)

## [수저O] 데이터 셋 서브 샘플링 & Train-Validation 나누기
- 아래의 코드를 참고하여 사용
- 반드시 Random Seed 42로 설정
- 25% Validation 으로 사용 권장

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.9, random_state=42)

## [수정 O] 학습 기반 분류기 설계
- 본인이 알고 있는 방법 중 가장 성능 좋은 분류기를 설계하시오

In [20]:
parameters = {'kernel':['rbf'], 'C': [1, 1.5, 2 , 5 , 10], 'gamma': [ 0.001, 0.0001,0.00005, 0.00001] ,'class_weight' : ['balanced']}
svc = svm.SVC(gamma="scale")
greed = GridSearchCV(svc, parameters, cv=2)
greed.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 1.5, 2, 5, 10], 'class_weight': ['balanced'],
                         'gamma': [0.001, 0.0001, 5e-05, 1e-05],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [21]:
greed.best_params_

{'C': 1.5, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}

In [0]:
y_predict = greed.predict(X_test)
confusion_matrix(y_test, y_predict)

In [0]:
target_names = ['dog', 'cat']
print(classification_report(y_test, y_predict, target_names=target_names))

              precision    recall  f1-score   support

         dog       0.61      0.64      0.63      9015
         cat       0.62      0.59      0.60      8986

    accuracy                           0.61     18001
   macro avg       0.61      0.61      0.61     18001
weighted avg       0.61      0.61      0.61     18001



## [수정O] 테스트 데이터 준비 
- 영상 32x32 로 Resize 후 1D Vector 로 변환 할 것


In [0]:
test = list()

datatest = "./input2/test1/test1/"
test_db= glob(datatest+'*.jpg')

for i in range(len(test_db)):

  img = Image.open(test_db[i])
  img = img.resize((32,32))
  img = np.array(img)/255
  img = img.reshape(-1)

  test.append(img)

test = np.array(test)
##scaler 추가
scaler.fit(test)
test = scaler.transform(test)

## [수정O] 학습한 모델을 이용하여 테스트 데이터의 라벨 구하기
- 라벨: 개(1), 고양이(0)

In [0]:
result = greed.predict(test)

## [수정O] 결과 파일로 저장하여 Kaggle 제출 준비
- 코드 변경 가능하나 기본 코드 사용을 권장함 

In [0]:
result = result.reshape(-1,1)

In [0]:

# numpy 를 Pandas 이용하여 결과 파일로 저장


import pandas as pd

print(result.shape)
df = pd.DataFrame(result, columns=["label"])
df.index = np.arange(1,len(df)+1)
df.index.name = 'id'

df.to_csv('results-yhchoi.csv',index=True, header=True)

(5000, 1)


## [수정O] Kaggle 에 result_yk.csv 파일 제출
- 제출 파일명 본인에게 맞게 수정
- "ykchoi-1810"" 본인의 이름과 제출 시간으로 수정

In [0]:
! kaggle competitions submit -c 2019-fall-pr-project -f results-yhchoi.csv -m "yhChoi-1031"

100% 33.1k/33.1k [00:02<00:00, 13.4kB/s]
Successfully submitted to InClass Competition at Sejong University